In [ ]:
import pandas as pd
import os
import chardet

# TXT 파일 경로 (변환할 파일 경로)
txt_file_path = r"C:\Users\TG\Documents\SF_DA_project\data\monthly_rent\서울특별시_전월세가_2021.txt"
csv_file_path = txt_file_path.replace(".txt", ".csv")

# 파일 인코딩 자동 감지 함수
def detect_encoding(file_path):
    with open(file_path, "rb") as f:
        raw_data = f.read(10000)  # 파일 일부만 읽어서 인코딩 감지
    result = chardet.detect(raw_data)
    return result["encoding"]

# 감지된 인코딩 적용
encoding_type = detect_encoding(txt_file_path)
print(f"감지된 파일 인코딩: {encoding_type}")

# TXT 파일 구분자 확인 후 적절한 delimiter 지정
with open(txt_file_path, "r", encoding=encoding_type) as file:
    first_line = file.readline()
    delimiter = "," if "," in first_line else "\t" if "\t" in first_line else " "  # 쉼표, 탭, 공백 중 선택

print(f" 감지된 구분자: '{delimiter}'")

# TXT 파일을 DataFrame으로 변환
df = pd.read_csv(txt_file_path, delimiter=delimiter, encoding=encoding_type)

# CSV 파일로 저장
df.to_csv(csv_file_path, index=False, encoding="utf-8-sig")

In [ ]:
import pandas as pd
import os
import glob
import chardet
import re

#CSV 파일이 들어있는 폴더 경로
folder_path = r"C:\Users\TG\Documents\SF_DA_project\data\monthly_rent"

# 폴더 내 모든 CSV 파일 찾기
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, "rb") as f:
        raw_data = f.read(10000)  # 파일 일부만 읽어서 인코딩 감지
    result = chardet.detect(raw_data)
    return result["encoding"]

data_list = []
for file in csv_files:
    encoding_type = detect_encoding(file)
    print(f"{file} 파일 인코딩 감지: {encoding_type}")
    
    try:
        df = pd.read_csv(file, encoding=encoding_type)
        data_list.append(df)
    except UnicodeDecodeError:
        print(f" {file}에서 {encoding_type} 인코딩 실패! UTF-8-SIG로 재시도")
        df = pd.read_csv(file, encoding="utf-8-sig", errors="replace")
        data_list.append(df)

df = pd.concat(data_list, ignore_index=True)

In [21]:
# 2. 전세를 월세로 변환하는 함수
def extract_contract_months(period):
    """ 계약 기간을 개월 수로 변환하는 함수 """
    if pd.isna(period):
        return 24  # 계약 기간이 없는 경우 기본 2년(24개월) 적용
    match = re.search(r'(\d{2})\.(\d{2})~(\d{2})\.(\d{2})', period)
    if match:
        start_year, start_month, end_year, end_month = map(int, match.groups())
        months = (end_year - start_year) * 12 + (end_month - start_month)
        return months
    return 24  # 기본값 2년

# 계약 개월 수 컬럼 추가
df["계약개월"] = df["계약기간"].apply(extract_contract_months)

# 법정 전환율 적용 (5.5%)
conversion_rate = 5.5 / 100  # 5.5% 연환산율
df["전환월세(만원)"] = (df["보증금(만원)"] * conversion_rate) / 12

In [22]:
# 3. 전세와 월세의 월세 비교 (자치구명 + 월세만 추출)
df_rent = df[["자치구명", "임대료(만원)"]].rename(columns={"임대료(만원)": "월세(만원)"})  # 기존 월세 데이터
df_jeonse = df[df["임대료(만원)"] == 0][["자치구명", "전환월세(만원)"]].rename(columns={"전환월세(만원)": "월세(만원)"})  # 전세 → 월세 변환된 데이터

# 두 데이터를 합쳐서 하나의 월세 데이터로 만들기
df_final = pd.concat([df_rent, df_jeonse], ignore_index=True)

In [ ]:
# 4. 지역별 평균 월세 계산
df_avg_rent = df_final.groupby("자치구명")["월세(만원)"].mean().reset_index()

df_avg_rent.to_csv("지역별_평균_월세.csv", index=False, encoding="utf-8-sig")

print(df_avg_rent.head())